In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Import Libraries
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

# Step 2: Dataset Preparation with Augmentation
data_dir = "/content/drive/MyDrive/flowers"

# Data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset = torchvision.datasets.ImageFolder(data_dir, transform=transform)
num_classes = len(dataset.classes)

# Step 3: Define Models
def build_model(model_name):
    if model_name == "efficientnet":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise ValueError("Invalid model name.")

    # Gradually unfreeze layers
    for param in model.parameters():
        param.requires_grad = False
    # Unfreeze the last 2 blocks
    if model_name == "efficientnet":
        for param in model.features[-2:].parameters():
            param.requires_grad = True
    else:
        for param in model.layer4.parameters():
            param.requires_grad = True

    return model

# Step 4: Few-Shot Training and Evaluation with Cross-Validation
def train_and_evaluate_with_kfold(model_name, dataset, k=5, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\n--- Fold {fold + 1}/{k} ---")
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=8, shuffle=False)

        model = build_model(model_name).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=0.001)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

        for epoch in range(epochs):
            # Training
            model.train()
            running_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            scheduler.step()
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        fold_accuracies.append(accuracy)
        print(f"Fold {fold + 1} Accuracy: {accuracy:.4f}")

    avg_accuracy = sum(fold_accuracies) / k
    print(f"\nAverage Accuracy for {model_name}: {avg_accuracy:.4f}")
    return avg_accuracy

# Step 5: Compare Models
efficientnet_accuracy = train_and_evaluate_with_kfold("efficientnet", dataset)
resnet50_accuracy = train_and_evaluate_with_kfold("resnet50", dataset)

# Step 6: Print Final Results
print(f"\nFinal Comparison:\nEfficientNet Accuracy: {efficientnet_accuracy:.4f}\nResNet-50 Accuracy: {resnet50_accuracy:.4f}")



--- Fold 1/5 ---


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 64.8MB/s]


Epoch [1/10], Loss: 1.4008
Epoch [2/10], Loss: 0.8248
Epoch [3/10], Loss: 0.6176
Epoch [4/10], Loss: 0.4265
Epoch [5/10], Loss: 0.4537
Epoch [6/10], Loss: 0.5441
Epoch [7/10], Loss: 0.5149
Epoch [8/10], Loss: 0.3978
Epoch [9/10], Loss: 0.4221
Epoch [10/10], Loss: 0.3270
Fold 1 Accuracy: 0.7778

--- Fold 2/5 ---
Epoch [1/10], Loss: 1.4897
Epoch [2/10], Loss: 0.9516
Epoch [3/10], Loss: 0.6848
Epoch [4/10], Loss: 0.5491
Epoch [5/10], Loss: 0.4977
Epoch [6/10], Loss: 0.4824
Epoch [7/10], Loss: 0.3607
Epoch [8/10], Loss: 0.3815
Epoch [9/10], Loss: 0.3188
Epoch [10/10], Loss: 0.2943
Fold 2 Accuracy: 0.7222

--- Fold 3/5 ---
Epoch [1/10], Loss: 1.4775
Epoch [2/10], Loss: 0.9514
Epoch [3/10], Loss: 0.6358
Epoch [4/10], Loss: 0.5880
Epoch [5/10], Loss: 0.6091
Epoch [6/10], Loss: 0.4431
Epoch [7/10], Loss: 0.3911
Epoch [8/10], Loss: 0.4334
Epoch [9/10], Loss: 0.3783
Epoch [10/10], Loss: 0.3627
Fold 3 Accuracy: 0.7778

--- Fold 4/5 ---
Epoch [1/10], Loss: 1.4923
Epoch [2/10], Loss: 0.8972
Epoch [

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 118MB/s]


Epoch [1/10], Loss: 1.2929
Epoch [2/10], Loss: 0.9470
Epoch [3/10], Loss: 1.1462
Epoch [4/10], Loss: 0.5928
Epoch [5/10], Loss: 0.5123
Epoch [6/10], Loss: 0.4440
Epoch [7/10], Loss: 0.3453
Epoch [8/10], Loss: 0.3113
Epoch [9/10], Loss: 0.2603
Epoch [10/10], Loss: 0.3780
Fold 1 Accuracy: 0.5000

--- Fold 2/5 ---
Epoch [1/10], Loss: 1.4009
Epoch [2/10], Loss: 1.0556
Epoch [3/10], Loss: 1.0487
Epoch [4/10], Loss: 0.7745
Epoch [5/10], Loss: 0.4711
Epoch [6/10], Loss: 0.5529
Epoch [7/10], Loss: 0.3793
Epoch [8/10], Loss: 0.3841
Epoch [9/10], Loss: 0.3695
Epoch [10/10], Loss: 0.3309
Fold 2 Accuracy: 0.6667

--- Fold 3/5 ---
Epoch [1/10], Loss: 1.4981
Epoch [2/10], Loss: 1.1678
Epoch [3/10], Loss: 0.6146
Epoch [4/10], Loss: 0.7081
Epoch [5/10], Loss: 0.6016
Epoch [6/10], Loss: 0.4933
Epoch [7/10], Loss: 0.2276
Epoch [8/10], Loss: 0.1908
Epoch [9/10], Loss: 0.2656
Epoch [10/10], Loss: 0.2068
Fold 3 Accuracy: 0.8333

--- Fold 4/5 ---
Epoch [1/10], Loss: 1.4364
Epoch [2/10], Loss: 1.2942
Epoch [